In [72]:
# wsl install (máy ảo)
# download ollama
# ollama run phi4
# data.csv

In [73]:
# 1. chunck data in case data so big (phân đoạn dữ liệu trong trường hợp dữ liệu quá lớn)
# 2. use ai to get code execute dataframe (sử dụng ai để lấy mã thực thi dataframe)
# 3. get resut from execute (lấy kết quả từ thực thi)
# 4. display to user interface (hiển thị đến giao diện người dùng)

# 1. chunck data in case data so big (phân đoạn dữ liệu trong trường hợp dữ liệu quá lớn)
# 2. use ai to get code execute dataframe (sử dụng ai để lấy mã thực thi dataframe)
# 3. get resut from execute (lấy kết quả từ thực thi)
# 4. display to user interface (hiển thị đến giao diện người dùng)

# TẠO AGENT

## LLAMA3.2

In [1]:
from together import Together
import os
import requests
import json

# Get a free API key from https://api.together.xyz/settings/api-keys
os.environ["TOGETHER_API_KEY"] = "36b75c9d516f765c779cabd44512da0fd57d5b560038d9079d564bf3d819f051"
# tgp_v1_Zmt7Q7H9iRYGmp-teZrDbjcUn7_FNWv-ZZM-9Is1Jpw
def llama32(messages, model_size=11):
  model = f"meta-llama/Llama-3.2-{model_size}B-Vision-Instruct-Turbo"
  url = "https://api.together.xyz/v1/chat/completions"
  payload = {
    "model": model,
    "max_tokens": 4096,
    "temperature": 0.0,
    "stop": ["<|eot_id|>","<|eom_id|>"],
    "messages": messages
  }

  headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.environ["TOGETHER_API_KEY"]
  }
  res = json.loads(requests.request("POST", url, headers=headers, data=json.dumps(payload)).content)

  if 'error' in res:
    raise Exception(res['error'])

  return res['choices'][0]['message']['content']

# GIẢI QUẾT VẤN ĐỀ

## Xác định keys

In [17]:
import pandas as pd

df = pd.read_csv("data.csv", encoding='utf-8').reset_index(drop=True)
names_counts = df.isnull().sum().sort_values(ascending=True)
dff = pd.DataFrame({'Name': names_counts.index.tolist(), 'Count': names_counts.values.tolist()})
keys = ""
for item in dff.head(38).Name.tolist()[1:]:
  keys += f"{item}, "
  
print(f"Từ {len(dff.head(38).Name.tolist())} cột thông tin\n")
keys

Từ 38 cột thông tin



'dbId, external_id, ElementId, Name, Category, CategoryId, Family Name, Workset, Outdoor Air per Area, Air Changes per Hour, Outdoor Air Method, Heating Set Point, Latent Heat Gain per person, Humidification Set Point, Cooling Set Point, Export to IFC, Specified Lighting Load per area, Specified Power Load per area, Infiltration Airflow per area, Plenum Lighting Contribution, Occupancy Schedule, Lighting Schedule, Power Schedule, Outdoor Air per Person, Dehumidification Set Point, Area per Person, Sensible Heat Gain per person, IfcGUID, Cost, Color, Description, Glow, Shininess, Smoothness, Transparency, Behavior, Class, '

# HỎI VÀ PHẢN HỒI

In [ ]:
question_client = "how many walls in data"

print(question_client)

question_me = [
    "I have column names (keys) in a raw aggregate .csv file\n" + 
    f'\nColumns: {keys}\n' + 
    f'\nWith question:\n' +
    f"\n{question_client}\n" +
    f"\nDo not create illustrative data, focus on 'column names'. Address the following requirements:"
    f"\n1. Search colums have the most relevant" +
    f"\n2. Solve the above question using dataframe\n"
    # "\n1. Tìm 3 key trong data keys liên quan nhiều nhất với câu hỏi \n"
    # "\n2. Lấy tất cả các key có liên quan và tạo thành 1 phần code python lấy kết quả với .csv dataframe\n"
    # "2. Tạo 3 phần code python lấy kết quả trong tài liệu Revit bằng API\n"
    ]

print()
print()
print(question_me[0])
print("*********************************************************")
messages = [
  {
    "role": "user",
    "content": question_me[0]
  }
]

response = llama32(messages)
print(response)

how many walls in data


I have column names (keys) in a raw aggregate .csv file

Columns: dbId, external_id, ElementId, Name, Category, CategoryId, Family Name, Workset, Outdoor Air per Area, Air Changes per Hour, Outdoor Air Method, Heating Set Point, Latent Heat Gain per person, Humidification Set Point, Cooling Set Point, Export to IFC, Specified Lighting Load per area, Specified Power Load per area, Infiltration Airflow per area, Plenum Lighting Contribution, Occupancy Schedule, Lighting Schedule, Power Schedule, Outdoor Air per Person, Dehumidification Set Point, Area per Person, Sensible Heat Gain per person, IfcGUID, Cost, Color, Description, Glow, Shininess, Smoothness, Transparency, Behavior, Class, 

With question:

how many walls in data

Do not create illustrative data, focus on 'column names'. Address the following requirements:
1. Search colums have the most relevant
2. Solve the above question using dataframe

*********************************************************
To

In [8]:
import pandas as pd

# Assuming df is your DataFrame
columns_to_search = ['Category']

# Search for the word "wall" in the specified columns
walls_count = df[columns_to_search].apply(lambda x: x.str.contains('wall', case=False).any(), axis=1).sum()

print(walls_count)

6


## PASS: How many families name in data ? Except: 506

In [83]:
question_client = "How many families name in data ?"

print(question_client)

question_me = [
    "I have column names (keys) in a raw aggregate .csv file\n" + 
    f'\nColumns: {keys}\n' + 
    f'\nWith question:\n' +
    f"\n{question_client}\n" +
    f"\nDo not create illustrative data, focus on 'column names'. Address the following requirements:"
    f"\n1. Find a columns in the data table related to the question" +
    f"\n2. Create a piece of code that solves the question above, using dataframe\n"
    ]

print()
print()
print(question_me[0])
print("*********************************************************")
messages = [
  {
    "role": "user",
    "content": question_me[0]
  }
]

response = llama32(messages)

import re

pattern = r"```python(.*?)```"
match = re.search(pattern, response, re.DOTALL)

if match:
    csharp_code = match.group(1).strip()
    print(csharp_code)
else:
    print("Không tìm thấy đoạn mã C# trong văn bản.")

How many families name in data ?


I have column names (keys) in a raw aggregate .csv file

Columns: dbId, external_id, ElementId, Name, Category, CategoryId, Family Name, Workset, Outdoor Air per Area, Air Changes per Hour, Outdoor Air Method, Heating Set Point, Latent Heat Gain per person, Humidification Set Point, Cooling Set Point, Export to IFC, Specified Lighting Load per area, Specified Power Load per area, Infiltration Airflow per area, Plenum Lighting Contribution, Occupancy Schedule, Lighting Schedule, Power Schedule, Outdoor Air per Person, Dehumidification Set Point, Area per Person, Sensible Heat Gain per person, IfcGUID, Cost, Color, Description, Glow, Shininess, Smoothness, Transparency, Behavior, Class, 

With question:

How many families name in data ?

Do not create illustrative data, focus on 'column names'. Address the following requirements:
1. Find a columns in the data table related to the question
2. Create a piece of code that solves the question above, using d

In [16]:
# assuming df is your DataFrame
family_names = df['Family Name'].unique()

df['Family Name'].sort_values().unique()

array(['01 - Existing', '02 - Demo', '3D View', '<In-session, Lighting>',
       '<Revision Schedule>', '<Revision Schedule> 2',
       'A10 - Substructure', 'ASTM A36', 'ASTM A500 - Grade B',
       'ASTM A500 - Grade B - Rectangular and Square',
       'ASTM A500 - Grade B - Round',
       'ASTM A500 - Grade C - Rectangular and Square',
       'ASTM A500 - Grade C - Round', 'ASTM A53', 'ASTM A572 - Grade 50',
       'ASTM A992', 'Acoustic Ceiling Tile', 'Acoustic Tile',
       'Active Storage', 'Active Storage - Hospital/Healthcare',
       'Actual HVAC Load', 'Actual Lighting - Dwelling Unit Load',
       'Actual Lighting Load', 'Actual Motor Load', 'Actual Other Load',
       'Actual Power Load', 'Actual Spare Load', 'Air',
       'Air/Train/Bus - Baggage Area', 'Airport - Concourse',
       'Alphanumeric', 'Aluminum', 'Architectural Elevation',
       'Architectural Plan', 'Architectural Section', 'Asphalt Shingle',
       'Atrium - Each Additional Floor', 'Atrium - First Three Fl

## PASS: How many revit categories ? 54

In [7]:
# question_client = df_ques.Question.values[80]
question_client = "How many floors in the current project?"

print(question_client)

question_me = [
    "I have column names (keys) in a raw aggregate .csv file\n" + 
    f'\nColumns: {keys}\n' + 
    f'\nWith question:\n' +
    f"\n{question_client}\n" +
    f"\nDo not create illustrative data, focus on 'column names'. Address the following requirements:"
    f"\n1. Find a columns in the data table related to the question" +
    f"\n2. Create a piece of code that solves the question above, using dataframe\n"
    ]

print()
print()
print(question_me[0])
print("*********************************************************")
messages = [
  {
    "role": "user",
    "content": question_me[0]
  }
]

response = llama32(messages)

import re

pattern = r"```python(.*?)```"
match = re.search(pattern, response, re.DOTALL)

if match:
    csharp_code = match.group(1).strip()
    print(csharp_code)
else:
    print("Không tìm thấy đoạn mã C# trong văn bản.")

How many floors in the current project?


I have column names (keys) in a raw aggregate .csv file

Columns: dbId, external_id, ElementId, Name, Category, CategoryId, Family Name, Workset, Outdoor Air per Area, Air Changes per Hour, Outdoor Air Method, Heating Set Point, Latent Heat Gain per person, Humidification Set Point, Cooling Set Point, Export to IFC, Specified Lighting Load per area, Specified Power Load per area, Infiltration Airflow per area, Plenum Lighting Contribution, Occupancy Schedule, Lighting Schedule, Power Schedule, Outdoor Air per Person, Dehumidification Set Point, Area per Person, Sensible Heat Gain per person, IfcGUID, Cost, Color, Description, Glow, Shininess, Smoothness, Transparency, Behavior, Class, 

With question:

How many floors in the current project?

Do not create illustrative data, focus on 'column names'. Address the following requirements:
1. Find a columns in the data table related to the question
2. Create a piece of code that solves the question 

In [ ]:
# Count the number of unique values in the 'Workset' column
num_floors = df['Workset'].nunique()
print(f'There are {num_floors} floors in the current project.')

There are 72 floors in the current project.


## FAIL: How many walls in data ? 6

In [95]:
question_client = df_ques.Question.values[80]
question_client = "How many walls in data ?"

print(question_client)

question_me = [
    "I have column names (keys) in a raw aggregate .csv file\n" + 
    f'\nColumns: {keys}\n' + 
    f'\nWith question:\n' +
    f"\n{question_client}\n" +
    f"\nDo not create illustrative data, focus on 'column names'. Address the following requirements:"
    f"\n1. Determine the question type"
    f"\n2. Find a columns in the data table related to the question" +
    f"\n3. Create a piece of code that solves the question above, using dataframe, case insensitive (ignore column names)\n"
  ]

print()
print()
print(question_me[0])
print("*********************************************************")
messages = [
  {
    "role": "user",
    "content": question_me[0]
  }
]

response = llama32(messages)

import re

pattern = r"```python(.*?)```"
match = re.search(pattern, response, re.DOTALL)

if match:
    csharp_code = match.group(1).strip()
    print(csharp_code)
else:
    print("Không tìm thấy đoạn mã C# trong văn bản.")

How many walls in data ?


I have column names (keys) in a raw aggregate .csv file

Columns: dbId, external_id, ElementId, Name, Category, CategoryId, Family Name, Workset, Outdoor Air per Area, Air Changes per Hour, Outdoor Air Method, Heating Set Point, Latent Heat Gain per person, Humidification Set Point, Cooling Set Point, Export to IFC, Specified Lighting Load per area, Specified Power Load per area, Infiltration Airflow per area, Plenum Lighting Contribution, Occupancy Schedule, Lighting Schedule, Power Schedule, Outdoor Air per Person, Dehumidification Set Point, Area per Person, Sensible Heat Gain per person, IfcGUID, Cost, Color, Description, Glow, Shininess, Smoothness, Transparency, Behavior, Class, 

With question:

How many walls in data ?

Do not create illustrative data, focus on 'column names'. Address the following requirements:
1. Determine the question type
2. Find a columns in the data table related to the question
3. Create a piece of code that solves the question

In [ ]:
# find the column related to the question (case insensitive)
category_column = [col for col in df.columns if 'category' in col.lower()]

walls = [wall for wall in df[category_column[0]] if 'revit walls' in wall.lower()]
walls
# # check if the column exists
# if category_column:
#     # count the number of unique values in the column
#     wall_count = 
#     print(f"There are {wall_count} walls in the data.")
# else:
#     print("No column related to the question found.")

# len(df[category_column[0]].unique())

['Revit Walls',
 'Revit Walls',
 'Revit Walls',
 'Revit Walls',
 'Revit Walls',
 'Revit Walls']

### sửa code

In [3]:
import pandas as pd

df = pd.read_csv("data.csv", encoding='utf-8').reset_index(drop=True)

wall_columns = ['Category']

# Initialize a counter for walls
wall_count = 0

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if any of the wall columns contain the word 'wall'
    if any(word in str(row[column]).lower() for column in wall_columns for word in ['wall', 'walls']):
        wall_count += 1

print(f"Number of walls: {wall_count}")

Number of walls: 6


# XỬ LÝ PHẢN HỒI LẤY MÃ THỰC THI

In [80]:
import re

text = response
# Sử dụng regular expression để tìm đoạn mã C# nằm giữa ```csharp và ```
pattern = r"```python(.*?)```"
match = re.search(pattern, text, re.DOTALL)

print(question_client, end='\n\n')

if match:
    csharp_code = match.group(1).strip()
    print(csharp_code)
else:
    print("Không tìm thấy đoạn mã C# trong văn bản.")
    

How many families name in data ?

import pandas as pd

# assuming df is your DataFrame
family_names = df['Family Name'].unique()

print(f"Number of unique family names: {len(family_names)}")


# TEST API KEY

In [ ]:
!pip install anthropic


In [ ]:
import anthropic

client = anthropic.Anthropic(
  # defaults to os.environ.get("ANTHROPIC_API_KEY")
  api_key=,
)

message_batch = client.beta.messages.batches.create(
    requests=[
        {
            "custom_id": "first-prompt-in-my-batch",
            "params": {
                "model": "claude-3-haiku-20240229",
                "max_tokens": 100,
                "messages": [
                    {
                        "role": "user",
                        "content": "Hey Claude, tell me a short fun fact about video games!",
                    }
                ],
            },
        },
        {
            "custom_id": "second-prompt-in-my-batch",
            "params": {
                "model": "claude-3-sonnet-20240229",
                "max_tokens": 100,
                "messages": [
                    {
                        "role": "user",
                        "content": "Hey Claude, tell me a short fun fact about bees!",
                    }
                ],
            },
        },
    ]
)
print(message_batch)

In [ ]:
import anthropic
import os
import time

# Thay thế YOUR_ANTHROPIC_API_KEY bằng API key thực tế của bạn
api_key = ""
client = anthropic.Anthropic(api_key=api_key)

message_batch = client.beta.messages.batches.create(
    requests=[
        {
            "custom_id": "first-prompt-in-my-batch",
            "params": {
                "model": "claude-3-haiku-20240229",
                "max_tokens": 1024,
                "messages": [
                    {
                        "role": "user",
                        "content": "Hey Claude, tell me a short fun fact about video games!",
                    }
                ],
            },
        }
    ]
)
print(f"Batch ID: {message_batch.id}")

while message_batch.processing_status == "in_progress":
    print("Batch is still processing...")
    time.sleep(10)  # Chờ một khoảng thời gian trước khi kiểm tra lại
    message_batch = client.beta.messages.batches.retrieve(message_batch.id)

print(f"Batch status: {message_batch.processing_status}")
if message_batch.processing_status == "succeeded":
    # Lấy kết quả từ results_url (nếu có) hoặc thông qua các phương thức khác
    print(f"Results URL: {message_batch.results_url}")
    # Bạn có thể cần gọi một API khác để lấy kết quả chi tiết của từng request
    # Ví dụ (có thể cần điều chỉnh theo API của Anthropic):
    # for request_result in client.beta.messages.batches.results(message_batch.id):
    #     print(f"Result for {request_result.custom_id}: {request_result.response.content[0].text}")
else:
    print(f"Batch failed or expired. Status: {message_batch.processing_status}")

print(message_batch)

Batch ID: msgbatch_01VCX4sVrhzLjGJA8EruVHVL
Batch is still processing...
Batch status: ended
Batch failed or expired. Status: ended
BetaMessageBatch(id='msgbatch_01VCX4sVrhzLjGJA8EruVHVL', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 3, 29, 18, 18, 23, 942097, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 3, 29, 18, 18, 25, 438944, tzinfo=TzInfo(UTC)), expires_at=datetime.datetime(2025, 3, 30, 18, 18, 23, 942097, tzinfo=datetime.timezone.utc), processing_status='ended', request_counts=BetaMessageBatchRequestCounts(canceled=0, errored=1, expired=0, processing=0, succeeded=0), results_url='https://api.anthropic.com/v1/messages/batches/msgbatch_01VCX4sVrhzLjGJA8EruVHVL/results', type='message_batch')


In [ ]:
!curl https://api.anthropic.com/v1/models \
  -H "X-API-Key: $ANTHROPIC_API_KEY"

{"type":"error","error":{"type":"authentication_error","message":"invalid x-api-key"}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    86  100    86    0     0    159      0 --:--:-- --:--:-- --:--:--   160
curl: (3) URL rejected: Bad hostname


In [ ]:

# BetaMessageBatch(
    # id='msgbatch_01YUwunUv7VdkXijrDX73AEY', 
    # archived_at=None, 
    # cancel_initiated_at=None, 
    # created_at=datetime.datetime(2025, 3, 29, 18, 9, 21, 911029, tzinfo=datetime.timezone.utc), 
    # ended_at=datetime.datetime(2025, 3, 29, 18, 9, 22, 996481, tzinfo=TzInfo(UTC)), 
    # expires_at=datetime.datetime(2025, 3, 30, 18, 9, 21, 911029, tzinfo=datetime.timezone.utc), 
    # processing_status='ended', 
    # request_counts=BetaMessageBatchRequestCounts(canceled=0, errored=2, expired=0, processing=0, succeeded=0), 
    # results_url='https://api.anthropic.com/v1/messages/batches/msgbatch_01YUwunUv7VdkXijrDX73AEY/results', 
    # type='message_batch')



# CHUẨN HÓA DỮ LIỆU CSV

In [183]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def chuan_hoa(df=DataFrame(), quantile='min'):
  '''
  Chuẩn hóa dữ liệu theo hàng với giá trị NaN
  param:
  df: DataFrame
  quantile: mean, min, 25%, 50%, 75%, max
  '''
  # df = df.astype(str)
  def remove_nan_list_comprehension(arr):
    return [(name_col, value) for [name_col, value] in arr if value != 'nan']

  data = []
  x, y = df.values.shape
  matrix = df.values

  for x in range(x):
    item = []
    num_nan = 0
    arr = np.column_stack((df.columns.values, matrix[x]))
    item.append(remove_nan_list_comprehension( arr ))

    data.append(item[0])

  # Chuyển mỗi danh sách tuple thành dictionary
  data_dicts = [dict(item) for item in data]

  # Tạo DataFrame
  df = pd.DataFrame(data_dicts)
  try:
    df.drop(columns='Unnamed: 0', inplace=True)
  except:
    pass
  mota = pd.DataFrame(df.isna().sum().sort_values(ascending=False).values).describe().to_dict()[0]

  # Đếm số lượng giá trị NaN của mỗi cột
  nan_counts = df.isna().sum()
  # print(nan_counts)

  # Xóa các cột có số lượng NaN > 5
  df_cleaned = df.loc[:, nan_counts <= int(mota[quantile])]

  return df_cleaned

df = pd.read_csv("data.csv", encoding='utf-8')
print(f'Data shape origin: {df.shape}')

quantile = '75%' # Thay đổi tỉ lệ chuẩn hóa ở đây
df = chuan_hoa(df, quantile=quantile)
print(f'Data shape processed with quantile({quantile}) : {df.shape}')
df.head(5)

Data shape origin: (804, 653)
Data shape processed with quantile(75%) : (804, 517)


,dbId,external_id,ElementId,Name,Category,CategoryId,Workset,Keynote,Export to IFC,IfcGUID,...,Apparent Load Phase C,Current Phase A,Current Phase B,Current Phase C,Neutral Rating,Neutral Bus,Ground Bus,Mounting Height,Placement Height,Family Name
0,40,9693c920-050e-4443-9b38-2491295576d1-00000000,0,A10 - Substructure [0],Revit Phases,-2000112.0,Phase Settings,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A10 - Substructure
1,43,9693c920-050e-4443-9b38-2491295576d1-00000002,2,01 - Existing [2],Revit Phases,-2000112.0,Phase Settings,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01 - Existing
2,787,1946603f-8ed1-4429-9d07-4a2981632975-000020cf,8399,02 - Demo [8399],Revit Phases,-2000112.0,Phase Settings,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02 - Demo
3,788,1946603f-8ed1-4429-9d07-4a2981632975-000020d1,8401,B20 - Exterior Enclosure [8401],Revit Phases,-2000112.0,Phase Settings,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B20 - Exterior Enclosure
4,789,f440c2b6-09c3-46da-b5c7-670feedae5cf-000020d6,8406,C10 - Interior Construction [8406],Revit Phases,-2000112.0,Phase Settings,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C10 - Interior Construction


In [19]:
df.iterrows()

<generator object DataFrame.iterrows at 0x00000161F515B9A0>

In [18]:
file_path = "data.csv"
df = pd.read_csv(file_path, encoding='utf-8')
wall_columns = ['Category']

# Initialize a counter for walls
wall_count = 0

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if any of the wall columns contain the word 'wall'
    if any(word in str(row[column]).lower() for column in wall_columns for word in ['wall', 'walls']):
        wall_count += 1
wall_count

6